In [1]:

import argparse
import sys
import os
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import collections
from scipy import misc


from scipy import misc
import numpy as np
import csv
import glob


# In[2]:


def bfs(map_visit, map_connect, ii):
    queue = collections.deque()
    queue.append((map_connect[ii], ii))

    union = []
    if not map_visit[ii]:
        union.append(ii)
        map_visit[ii] = True
    while queue:
        connects, ii = queue.popleft()
        for connect in connects:
            if not map_visit[connect]:
                queue.append((map_connect[connect], connect))
                map_visit[connect] = True
                union.append(connect)
    return union

def plot_save(image, filename):
    fig = plt.figure(dpi=300)
    ax = fig.add_subplot(111)
    ax.imshow(image, aspect='auto', cmap=plt.cm.gray, interpolation='nearest')
    plt.savefig(filename)

def get_paths(lfw_dir, pairs):
    nrof_skipped_pairs = 0
    path_list = []
    issame_list = []
    # add new rule, pair = 5
    for pair in pairs:
        if len(pair) == 3:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[1]))
            path1 = add_extension(os.path.join(lfw_dir, pair[0], pair[2]))
            issame = True
        elif len(pair) == 4:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[1]))
            path1 = add_extension(os.path.join(lfw_dir, pair[2], pair[3]))
            issame = False
        elif len(pair) == 5:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[1]))
            path1 = add_extension(os.path.join(lfw_dir, pair[2], pair[3]))
            if pair[4] == 'T':
                issame = True
            elif pair[4] == 'F':
                issame = False
        if os.path.exists(path0) and os.path.exists(path1):  # Only add the pair if both paths exist
            path_list += (path0, path1)
            issame_list.append(issame)
        else:
            nrof_skipped_pairs += 1
    if nrof_skipped_pairs > 0:
        print('Skipped %d image pairs' % nrof_skipped_pairs)

    return path_list, issame_list


def add_extension(path):
    if os.path.exists(path+'.jpg'):
        return path+'.jpg'
    elif os.path.exists(path+'.png'):
        return path+'.png'
    else:
        raise RuntimeError('No file "%s" with extension png or jpg.' % path)

def read_pairs(pairs_filename):
    pairs = []
    with open(pairs_filename, 'r') as f:
        for line in f.readlines()[1:]:
            pair = line.strip().split()
            pairs.append(pair)
    return np.array(pairs)


In [2]:
lfw_pairs='/root/facenet_davidsandberg/data/icon_valv2.txt'
lfw_dir='/root/icon180k_image_160/'

pairs = read_pairs(os.path.expanduser(lfw_pairs))
print(pairs[0:6])
# Get the paths for the corresponding images
lfw_paths, actual_issame = get_paths(os.path.expanduser(lfw_dir), pairs)
print(lfw_paths[0:8])



[['1449011786' '0' '1095451819' '0' 'T']
 ['1448039576' '0' '1185423527' '0' 'F']
 ['1447550782' '0' '1462095693' '0' 'T']
 ['1458379276' '0' '1381475296' '0' 'F']
 ['1447550782' '0' '1230636915' '0' 'T']
 ['1447059340' '0' '1462029352' '0' 'F']]
['/root/icon180k_image_160/1449011786/0.png', '/root/icon180k_image_160/1095451819/0.png', '/root/icon180k_image_160/1448039576/0.png', '/root/icon180k_image_160/1185423527/0.png', '/root/icon180k_image_160/1447550782/0.png', '/root/icon180k_image_160/1462095693/0.png', '/root/icon180k_image_160/1458379276/0.png', '/root/icon180k_image_160/1381475296/0.png']


In [3]:
# create icon validation for npair

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 12 17:34:25 2019

@author: Cheng-Hung
"""

import os
import tarfile
import subprocess
import numpy as np
from scipy.io import loadmat
import h5py
import fuel
from fuel.datasets.hdf5 import H5PYDataset
import cv2
from tqdm import tqdm


def preprocess(hwc_bgr_image, size):
    hwc_rgb_image = cv2.cvtColor(hwc_bgr_image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(hwc_rgb_image, (size))
    chw_image = np.transpose(resized, axes=(2, 0, 1))
    return chw_image




In [5]:
    dataset_name = "cars196"
    archive_basename = "car_ims"

    fuel_root_path = "./data"
#     fuel_root_path = "./datasets/data"
    fuel_data_path = os.path.join(fuel_root_path, dataset_name)
#     image_filepath = os.path.join(fuel_data_path, archive_basename + ".tar.gz")
#     label_filepath = os.path.join(fuel_data_path, "cars_annos.mat")

#     # Extract car_ims.tgz if car_ims directory does not exist
#     with tarfile.open(image_filepath, "r") as tf:
#         jpg_filenames = [fn for fn in tf.getnames() if fn.endswith(".jpg")]
#     jpg_filenames.sort()
#     num_examples = len(jpg_filenames)  # ????
#     if not os.path.exists(os.path.join(fuel_data_path, archive_basename)):
#         subprocess.call(["tar", "zxvf", image_filepath.replace("\\", "/"),
#                          "-C", fuel_data_path.replace("\\", "/"),
#                          "--force-local"])
jpg_filenames = lfw_paths
num_examples = len(jpg_filenames)

In [7]:
    # Extract class labels
#     cars_annos = loadmat(label_filepath)
#     annotations = cars_annos["annotations"].ravel()
#     annotations = sorted(annotations, key=lambda a: str(a[0][0]))
    annotations = range(num_examples)
    class_labels = []
    for annotation in annotations:
#         class_label = int(annotation[5])
        class_labels.append(annotation)

print(class_labels[0:4])

[0, 1, 2, 3]


In [8]:
    # open hdf5 file
    hdf5_filename = "cars196.hdf5"
    hdf5_filepath = os.path.join(fuel_data_path, hdf5_filename)
    hdf5 = h5py.File(hdf5_filepath, mode="w")

    # store images
    image_size = (256, 256)
    array_shape = (num_examples, 3) + image_size
    ds_images = hdf5.create_dataset("images", array_shape, dtype=np.uint8)
    ds_images.dims[0].label = "batch"
    ds_images.dims[1].label = "channel"
    ds_images.dims[2].label = "height"
    ds_images.dims[3].label = "width"



In [9]:
    # write images to the disk
    for i, filename in tqdm(enumerate(jpg_filenames), total=num_examples,
                            desc=hdf5_filepath):
        raw_image = cv2.imread( filename,
                               cv2.IMREAD_COLOR)  # BGR image
        image = preprocess(raw_image, image_size)
        ds_images[i] = image
        

./data/cars196/cars196.hdf5: 100%|██████████| 6000/6000 [00:08<00:00, 688.01it/s]


In [10]:
    # store the targets (class labels)
    targets = np.array(class_labels, np.int32).reshape(num_examples, 1)
    ds_targets = hdf5.create_dataset("targets", data=targets)
    ds_targets.dims[0].label = "batch"
    ds_targets.dims[1].label = "class_labels"



In [11]:
    # specify the splits (labels 1~98 for train, 99~196 for test)
    test_head = class_labels.index(0)
    print(test_head)
    split_train, split_test = (0, test_head), (test_head, num_examples)
    split_dict = dict(train=dict(images=split_train, targets=split_train),
                      test=dict(images=split_test, targets=split_test))
    hdf5.attrs["split"] = H5PYDataset.create_split_array(split_dict)



0


In [12]:
    hdf5.flush()
    hdf5.close()